In [1]:
import pandas as pd
import json

dashboard = pd.ExcelFile("data.xlsx")

In [2]:
data = pd.read_excel(dashboard,"Statistics")

aes = pd.read_excel(dashboard,"AEs")

referrals = pd.read_excel(dashboard, "Referrals")

partnerDistricts = pd.read_excel("partnerDistrict.xlsx")

In [3]:
data['totalMCs'] = data['mc15-19'] + data['mc20-24'] +data['mc25-29']+data['mc30-34']+data['mc35-39'] + data['mc40-44'] + data['mc45-49']+ data['mc50']

data

,AERecordingMonth,AERecordingSite,AERecordingYear,District,LinkagesSelection,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,...,totalhivNegative,totalhivNegativeNC,uncircumcisedClientsForHTS,uncircumcised_care,uncircumcised_prep,uncircumcised_srh,uncircumcised_sti,vmmc_number,year,facilityType
0,0,0,0,HURUNGWE,0.0,0,0,0,TENGWE - 101886 - CLINIC,12.0,...,6,0,0.0,0,0,0,0,0,2025,0
1,0,0,0,HURUNGWE,0.0,0,0,0,TENGWE - 101886 - CLINIC,11.0,...,0,0,0.0,0,0,0,0,0,2025,0
2,0,0,0,HURUNGWE,NaN,0,0,0,TENGWE - 101886 - CLINIC,23.0,...,0,0,NaN,0,0,0,0,0,2025,0
3,0,0,0,HURUNGWE,0.0,0,0,0,TENGWE - 101886 - CLINIC,27.0,...,3,0,0.0,0,0,0,0,0,2025,0
4,0,0,0,HURUNGWE,0.0,0,0,0,TENGWE - 101886 - CLINIC,5.0,...,0,0,0.0,0,0,0,0,0,2025,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,0,0,0,HURUNGWE,0.0,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,11.0,...,8,0,0.0,0,0,0,0,0,2025,Outreach
136,0,0,0,HURUNGWE,0.0,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,8.0,...,6,0,0.0,0,0,0,0,0,2025,Outreach
137,0,0,0,HURUNGWE,0.0,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,18.0,...,13,0,0.0,0,0,0,0,0,2025,Outreach
138,0,0,0,HURUNGWE,0.0,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,9.0,...,7,0,0.0,0,0,0,0,0,2025,Outreach


In [4]:
aes

,AERecordingMonth,AERecordingSite,AERecordingYear,District,aeComments,ae_classification,ae_type_code,circumcising_cadre,client_age,date_ae_identified,mcMethod,vmmc_number
0,June,Makumbe Hospital,submission,GOROMONZI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,June,Dombotombo Clinic,submission,MARONDERA,Wound care counseling and elevations emphasized,mild,SW,Nurse,15.0,2025-06-25,Shangring,132.0
2,February,Border Church Clinic,submission,MARONDERA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,February,Turf Clinic,2025,MHONDORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,December,Donain,2024,MHONDORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,March,Karoi District Hospital,2025,HURUNGWE,Encouraged to elevate penis to promote venous ...,Moderate,Swelling/ moderate,Nurse,16.0,2025-03-05,Surgical,68.0


In [5]:
# Ensure Month-Year columns are strings for easy comparison
data['year'] = data['year'].astype(str)
aes['AERecordingYear'] = aes['AERecordingYear'].astype(str)

data['recordingMonth'] = data['recordingMonth'].astype(str)
aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)

# Transform df1 to include the list object
json_data = []

for _, row in data.iterrows():
    row_data = row.to_dict()
    print(type(row['totalAES']))
    if row['totalAES'] >= 1:
        # Filter df2 to find matching rows
        recordedAEs = aes[
            (aes['AERecordingSite'] == row['facilityName']) &
            (aes['AERecordingMonth'] == row['recordingMonth']) &
            (aes['AERecordingYear'] == row['year'])
        ]

        print(recordedAEs)
        
        # Add list object to the row if matches are found
        row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
    else:
        # Add an empty list if no matches
        row_data['matchingAES'] = []


    if row['referralToOtherServices'] >= 1:
        # Filter df2 to find matching rows
        otherReferrals = referrals[
            (referrals['ReferrelRecordingSite'] == row['facilityName']) &
            (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
            (referrals['ReferrelRecordingYear'] == row['year'])
        ]


        # Add list object to the row if matches are found
        row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
    else:
        # Add an empty list if no matches
        row_data['otherReferrals'] = []
    
    # Append to the final JSON structure
    json_data.append(row_data)

# Convert to JSON
result_json = json.dumps(json_data, indent=4)

output_file = "data.json"

# Save the JSON data to the file
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(result_json)

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
Empty DataFrame
Columns: [AERecordingMonth, AERecordingSite, AERecordingYear, District, aeComments, ae_classification, ae_type_code, circumcising_cadre, client_age, date_ae_identified, mcMethod, vmmc_number]
Index: []
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
Empty DataFrame
Columns: [AERecordingMonth, AERecordingSite, AERecordingYear, District, aeComments, ae_classification, ae_type_code, circumcising_cadre, client_age, date_ae_identified, mcMethod, vmmc_number]
Index: []
<class 'int'>
<class 'int'>
  AE